# Conversión de un modelo de Caffe a OpenVINO™

**Ejemplo Práctico**

## Pre-requisitos
Como siempre, primero se debe activar las variables de entorno de OpenVINO, **ejecutando** la correspondiente celda de código.
- *Windows*

In [9]:
%%cmd
"C:\Program Files (x86)\Intel\openvino_2021\bin\setupvars.bat"

Microsoft Windows [Version 10.0.19043.1165]
(c) Microsoft Corporation. All rights reserved.

C:\Users\josej\Documents\Workshop-ESPEL-MCT-2021\Secci¢n 3>"C:\Program Files (x86)\Intel\openvino_2021\bin\setupvars.bat"
Python 3.8.10
[setupvars.bat] OpenVINO environment initialized

C:\Users\josej\Documents\Workshop-ESPEL-MCT-2021\Secci¢n 3>

- *Linux*

In [1]:
%%bash
source /opt/intel/openvino_2021/bin/setupvars.sh

[setupvars.sh] OpenVINO environment initialized


## Descargar el archivo de Conversión
El presente archivo de ejemplo corresponde a un modelo para Detección de Objetos de la arquitectura Single Shot Detectors(SSD) llamado SSD300[1], es capaz de reconocer entre 20 clases diferentes del Dataset Pascal VOC2007. Para este ejemplo, se presenta el uso de la herramienta de descarga de modelos de OpenVINO llamada `model downloader`, dicha herramienta puede descargar modelos pre-entrenados ya convertidos de Intel o modelos públicos para ser convertidos, es un script de Python que necesita el nombre de modelo y a  para continuar con este tutorial, se recomienda **ejecutar** esta celda de código 
- *Windows*

In [6]:
%%cmd
SET dir_md="C:\Program Files (x86)\Intel\openvino_2021\deployment_tools\tools\model_downloader\downloader.py" 
SET nombre_modelo="ssd300"
python %dir_md% --name %nombre_modelo%


Microsoft Windows [Version 10.0.19043.1165]
(c) Microsoft Corporation. All rights reserved.

C:\Users\josej\Documents\Workshop-ESPEL-MCT-2021\Secci¢n 3>SET dir_md="C:\Program Files (x86)\Intel\openvino_2021\deployment_tools\tools\model_downloader\downloader.py" 

C:\Users\josej\Documents\Workshop-ESPEL-MCT-2021\Secci¢n 3>SET nombre_modelo="ssd300"

C:\Users\josej\Documents\Workshop-ESPEL-MCT-2021\Secci¢n 3>python %dir_md%  --name %nombre_modelo%
################|| Downloading ssd300 ||################

========== Downloading C:\Users\josej\Documents\Workshop-ESPEL-MCT-2021\Sección 3\public\ssd300\ssd300.tar.gz
... 1%, 1024 KB, 181 KB/s, 5 seconds passed
... 2%, 2048 KB, 197 KB/s, 10 seconds passed
... 3%, 3072 KB, 224 KB/s, 13 seconds passed
... 4%, 4096 KB, 221 KB/s, 18 seconds passed
... 5%, 5120 KB, 240 KB/s, 21 seconds passed
... 6%, 6144 KB, 253 KB/s, 24 seconds passed
... 7%, 7168 KB, 261 KB/s, 27 seconds passed
... 8%, 8192 KB, 253 KB/s, 32 seconds passed
... 9%, 9216 KB, 255 KB

- *Linux*

In [2]:
%%bash  
dir_md=/opt/intel/openvino_2021/deployment_tools/tools/model_downloader/downloader.py
nombre_modelo="ssd300"
python $dir_md --name $nombre_modelo

################|| Downloading ssd300 ||################

========== Downloading /home/josejacomeb/Documents/Workshop-ESPEL-MCT-2021/Sección 3/public/ssd300/ssd300.tar.gz
... 1%, 1024 KB, 259 KB/s, 3 seconds passed
... 2%, 2048 KB, 247 KB/s, 8 seconds passed
... 3%, 3072 KB, 214 KB/s, 14 seconds passed
... 4%, 4096 KB, 217 KB/s, 18 seconds passed
... 5%, 5120 KB, 214 KB/s, 23 seconds passed
... 6%, 6144 KB, 215 KB/s, 28 seconds passed
... 7%, 7168 KB, 216 KB/s, 33 seconds passed
... 8%, 8192 KB, 211 KB/s, 38 seconds passed
... 9%, 9216 KB, 154 KB/s, 59 seconds passed
... 10%, 10240 KB, 134 KB/s, 76 seconds passed
... 11%, 11264 KB, 125 KB/s, 89 seconds passed
... 12%, 12288 KB, 123 KB/s, 99 seconds passed
... 13%, 13312 KB, 122 KB/s, 109 seconds passed
... 15%, 14336 KB, 121 KB/s, 118 seconds passed
... 16%, 15360 KB, 122 KB/s, 125 seconds passed
... 17%, 16384 KB, 123 KB/s, 132 seconds passed
... 18%, 17408 KB, 125 KB/s, 138 seconds passed
... 19%, 18432 KB, 125 KB/s, 147 seconds pass

En este caso, no es necesaria la descompresión pues el mismo programa descomprime este modelo en la ruta `public\ssd300\models\VGGNet\VOC0712Plus\SSD_300x300_ft`, a continuación solo basta convertir el modelo a la Representación Intermedia.

## Conversión de un Modelo Caffe hacia la Representación Intermedia
La carpeta descomprimida comprende de varios archivos, entre los más importantes:
- deploy.prototxt 
- VGG_VOC0712Plus_SSD_300x300_ft_iter_160000.caffemodel
- solver.prototxt
- test.prototxt y train.prototxt

En este específico ejemplo de Caffe, el archivo *deploy.txt* contiene información sobre la arquitectura final del modelo SSD, en cambio el archivo *VGG_VOC0712Plus_SSD_300x300_ft_iter_160000.caffemodel* comprende una descripción binaria de los pesos del modelo, el archivo *solver.prototxt* contiene información sobre los parámetros de entrenamiento que se usaron con esa red y los archivos *test.prototxt* y *train.protoxt* son archivos que contienen la arquitectura y se usaron para el entrenamiento y validación.

El comando mínimo para la conversión de un modelo de Caffe es el siguiente[2]: 
```bash
python3 mo.py --input_model <INPUT_MODEL>.caffemodel --output_dir <OUTPUT_MODEL_DIR>
```

El argumento *INPUT_MODEL* se refiere a la ubicación del archivo de los pesos del modelo, en formato caffemodel. La variable *OUTPUT_MODEL_DIR* es un directorio donde deseemos guardar nuestro modelo convertido

Debido a que este modelo es de Detección de Objetos, es necesario además añadir la descripción de la arquitectura[2], este archivo es el ya mencionado *deploy.protxt* que se ejecuta con el argumento *--input_proto* en total el comando total a utilizar es el siguiente: 
```bash
python3 mo.py --input_model <SAVED_MODEL_DIR> --input_proto <PIPELINE_FILE> --output_dir <OUTPUT_DIR> 
```

La conversión se puede realizar **ejecutando** el siguiente script en este Jupyter Notebook.

In [12]:
import os, sys
nombre_mo = "mo.py"
ruta_modelo = "public/ssd300/models/VGGNet/VOC0712Plus/SSD_300x300_ft"
carpeta_salida_archivos = "ssd300_OpenVINO" #Cambiar esta ruta si se desea
nombre_archivo_pesos="VGG_VOC0712Plus_SSD_300x300_ft_iter_160000.caffemodel"
nombre_archivo_proto = "deploy.prototxt"
ruta_modelo_entrada = os.path.join(ruta_modelo, nombre_archivo_pesos)
ruta_archivo_proto = os.path.join(ruta_modelo, nombre_archivo_proto)
if sys.platform == "win32":
    ruta_mo = os.path.normpath("C:/Program Files (x86)/Intel/openvino_2021/deployment_tools/model_optimizer")
    ruta_model_optimizer = os.path.join(ruta_mo, nombre_mo) #Union de directorios 
    ruta_model_optimizer = '"{}"'.format(ruta_model_optimizer) #Conversion de Path a formato Windows
    comando_total = ruta_model_optimizer + " --input_model " + ruta_modelo_entrada \
    + " --input_proto " + ruta_archivo_proto  + " --output_dir " + carpeta_salida_archivos 
    !python {comando_total}
    print("Archivo Convertido Guardado en: {}".format(carpeta_salida_archivos))
elif sys.platform == "linux":
    ruta_mo = "/opt/intel/openvino_2021/deployment_tools/model_optimizer/"
    ruta_model_optimizer = os.path.join(ruta_mo, nombre_mo) #Union de directorios 
    comando_total = ruta_model_optimizer + " --input_model " + ruta_modelo_entrada \
    + " --input_proto " + ruta_archivo_proto  + " --output_dir " + carpeta_salida_archivos 
    !python {comando_total}
    print("Archivo Convertido Guardado en: {}".format(carpeta_salida_archivos))
else:
    print("SO no soportado")

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	C:\Users\josej\Documents\Workshop-ESPEL-MCT-2021\Sección 3\public/ssd300/models/VGGNet/VOC0712Plus/SSD_300x300_ft\VGG_VOC0712Plus_SSD_300x300_ft_iter_160000.caffemodel
	- Path for generated IR: 	C:\Users\josej\Documents\Workshop-ESPEL-MCT-2021\Sección 3\ssd300_OpenVINO
	- IR output name: 	VGG_VOC0712Plus_SSD_300x300_ft_iter_160000
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	Not specified, inherited from the model
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	None
	- Reverse input channels: 	False
Caffe specific parameters:
	- Path to Python Caffe* parser generated fro

[ WARNING ]  Using fallback to produce IR.


Dependiendo de la velocidad de ejecución del computador, se demorará en promedio *un minutos* en realizar la conversión.

Si la conversión ha sido exitosa, se mostrará un mensaje parecido al siguiente que indica la ruta de los archivos convertidos

```bash
[ SUCCESS ] Generated IR version 10 model.
[ SUCCESS ] XML file: C:\Users\josej\Documents\Workshop-ESPEL-MCT-2021\Sección 3\ssd300_OpenVINO\VGG_VOC0712Plus_SSD_300x300_ft_iter_160000.xml
[ SUCCESS ] BIN file: C:\Users\josej\Documents\Workshop-ESPEL-MCT-2021\Sección 3\ssd300_OpenVINO\VGG_VOC0712Plus_SSD_300x300_ft_iter_160000.bin
[ SUCCESS ] Total execution time: 14.76 seconds. 
Archivo Guardado en: ssd300_OpenVINO
```

## Referencias
- [1] Intel. (2021, Agosto 31). ssd300. Intel. [https://docs.openvinotoolkit.org/latest/omz_models_model_ssd300.html](https://docs.openvinotoolkit.org/latest/omz_models_model_ssd300.html)
- [1] Intel. (2021, Agosto 31). Converting a Caffe* Model . Intel. [https://docs.openvinotoolkit.org/latest/openvino_docs_MO_DG_prepare_model_convert_model_Convert_Model_From_Caffe.html](https://docs.openvinotoolkit.org/latest/openvino_docs_MO_DG_prepare_model_convert_model_Convert_Model_From_Caffe.html)
